In [1]:
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import seaborn as sns
from subprocess import check_output
# from seedlings_resnet_50 import resnet50_model
from seedlings_resnet_50_AUG import resnet50_model

from PIL import Image
%matplotlib inline


Using TensorFlow backend.


In [2]:
fpa = []
fpa.append(check_output(["ls", "seedlings_data/test/"]).decode("utf8").strip().split("\n"))

filez = np.array(fpa)

fpa = np.array(fpa)
fpa = np.core.defchararray.add("seedlings_data/test/", fpa)
fpa = fpa.squeeze()
fpa.shape

(794,)

In [3]:
imgs = []

for im_path in fpa:
    if 'Thumbs.db' not in im_path:
        #print(im_path)
        imageCV = cv2.imread(im_path)
        imgs.append(cv2.resize(imageCV, (300, 300), interpolation=cv2.INTER_AREA))

imgs = np.array(imgs)
imgs.shape

(794, 300, 300, 3)

In [4]:
classes = check_output(["ls", "seedlings_data/train"]).decode("utf8").strip().split("\n")

In [5]:
img_rows, img_cols = 300, 300  # Resolution of inputs
channel = 3
num_classes = 12
batch_size = 32
epochs = 2

In [6]:
# submModel = resnet50_model(img_rows, img_cols, channel, num_classes)
submModel = resnet50_model(img_rows, img_cols, channel, num_classes)
preds = submModel.predict(imgs, batch_size=32, verbose=1)
print(preds.shape)
preds

/home/michal/PycharmProjects/cnn_finetune/seedlings_resnet_50_AUG.py:82: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, shortcut], mode='sum')
/usr/local/lib/python3.5/dist-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


/home/michal/PycharmProjects/cnn_finetune/seedlings_resnet_50_AUG.py:43: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  x = merge([x, input_tensor], mode='sum')


Dimension ordering (tf for Tensorflow): tf
Loading NOT FRESH weights from file: seedl_chkp/seedl_wgh_AUG_lr9e-4_rot7-ep28-0.97068.hdf5


Loading NOT FRESH weights finished...


 32/794 [>.............................] - ETA: 34s

...

736/794 [==========================>...] - ETA: 0s

768/794 [============================>.] - ETA: 0s

794/794 [==============================] - 7s 8ms/step


(794, 12)


array([[  5.40488225e-04,   1.82685224e-04,   8.86816488e-05, ...,
          1.35060842e-03,   9.95833635e-01,   9.77149102e-05],
       [  1.39484243e-07,   4.95278947e-08,   7.84441099e-08, ...,
          4.08677669e-09,   3.13708512e-07,   6.15211661e-07],
       [  4.62480818e-11,   1.10788223e-09,   1.51923231e-07, ...,
          5.65272412e-11,   8.82880661e-12,   9.99999762e-01],
       ..., 
       [  6.66785382e-09,   2.10262918e-08,   1.22083918e-07, ...,
          3.82653687e-09,   9.30529709e-10,   9.99998093e-01],
       [  2.03476546e-07,   9.99975920e-01,   1.70632404e-06, ...,
          2.80447193e-06,   4.84883856e-07,   1.91980320e-07],
       [  1.58273378e-05,   1.32258492e-07,   2.56604178e-08, ...,
          1.39678408e-11,   2.90771562e-09,   2.53031462e-08]], dtype=float32)

In [7]:
classes = np.array(classes)
classes.shape
max_idxes = np.argmax(preds, axis=1)
max_idxes.shape

pred_class = classes[max_idxes]
pred_class = np.expand_dims(pred_class, axis=0)
print(pred_class.shape)
print(filez.shape)

conc = np.concatenate((filez, pred_class), axis=0)
conc = conc.astype('str')
print(conc.shape)
conc


(1, 794)
(1, 794)
(2, 794)


array([['0021e90e4.png', '003d61042.png', '007b3da8b.png', ...,
        'fef2ade8c.png', 'ff65bc002.png', 'ffc6f8527.png'],
       ['Small-flowered Cranesbill', 'Fat Hen', 'Sugar beet', ...,
        'Sugar beet', 'Charlock', 'Common wheat']],
      dtype='<U25')

In [8]:
res = np.apply_along_axis(lambda d: d[0] + ',' + d[1], 0, conc)

res.tofile("seedl_subms/seedl_wgh_AUG_lr9e-4_rot7-ep28-0.97068.csv", sep='\n')
#np.savetxt("seedl_subms/subm94.csv", conc, delimiter=",")